In [7]:
import torch
import numpy as np
from models.training import easyTrainer, weights_to_dataset
from models.nODE import nODE, make_nODE_from_parameters
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset
from torchdiffeq import odeint
from torch.utils.data import DataLoader 
from data_creation import create_dataset, torch_parameter_structure

In [8]:
class NetworkDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)
    
    def __getitem__(self,idx):
        return self.x[idx], self.y[idx]

In [ ]:
x_exact, x_train, y_train, param = create_dataset(2,1,1000)

trainData = NetworkDataset(x_train,y_train)
loader = DataLoader(trainData, batch_size=64, shuffle=True)

In [32]:
class param_classifier(torch.nn.Module):
    def __init__(self, input_size, ode_dim, layers_size=[10, 10], device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')):
        super().__init__()

        self.layers_size = layers_size
        self.input_size = input_size
        self.output_size = 2 * ode_dim ** 2 + 3 * ode_dim
        # print(self.output_size)
        self.ode_dim = ode_dim

        self.num_layers = len(layers_size) + 1
        self.device = device

        # initialise linear layers for classification block
        self.linears = torch.nn.ModuleList([torch.nn.Linear(input_size, layers_size[0])])
        self.linears.extend(
            [torch.nn.Linear(layers_size[i - 1], layers_size[i]) for i in range(1, self.num_layers - 1)])
        self.linears.append(torch.nn.Linear(layers_size[-1], self.output_size))
        return
    

    def forward_integration(self, x, parameter, integration_time=None):
        if integration_time is None:
            integration_interval = torch.tensor([0., 1.]).type_as(x)
        else:
            integration_interval = torch.tensor(integration_time).type_as(x)

        par_struct = torch_parameter_structure(self.ode_dim)
        par_struct.set_vec_par(parameter)
        gamma, Win, bin, Wout, bout = par_struct.get_parameters()
        
        dt = 0.01
        out = odeint(lambda t, x : self.right_hand_side(t, x, Win, Wout, bin, bout, gamma), x, integration_interval, method='euler', options={'step_size': dt})
        return out[-1, :]
               
    def right_hand_side(self, t, x, Win,Wout,bin,bout,gamma):
        out = x.matmul(Win.t()) + bin.t()
        out = out.matmul(Wout.t()) + bout.t()
        out = x.matmul(gamma) + out
        return out

    def get_adjacency(self,parameter):
        Win = torch.zeros(self.ode_dim,self.ode_dim).to(self.device)
        Wout = torch.zeros(self.ode_dim,self.ode_dim).to(self.device)

        k=0

        for i in range(0,self.ode_dim):
            for j in range(0,self.ode_dim):
                Win[i][j] = parameter[k]
                k += 1

        for i in range(0,self.ode_dim):
            for j in range(0,self.ode_dim):
                Wout[i][j] = parameter[k]
                k += 1
        
        A = Wout.matmul(Win).flatten()
        return A

    # forward pass of NN (both classifier and neural ODE)
    def forward(self, data):
        x = data
        for i in range(0, self.num_layers):
            x = self.linears[i](x)
            if i < self.num_layers-1:
                x = F.relu(x)

        # here x denote the estimated parameters for the ODE
        # x = self.linears[len(self.layers_size) - 1](x)

        return x

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

classifier = param_classifier(8, 2).to(device)

x = x_train[0][0].to(device)

out = classifier(x)

u0 = x[:2]

print(out)

ut_hat = classifier.forward_integration(u0, out)

print(ut_hat)



tensor([-0.1459, -0.0978, -0.0176, -0.0727, -0.0733, -0.1363,  0.0458, -0.0753,
         0.1888, -0.0271,  0.1033, -0.2813,  0.1313,  0.2141], device='cuda:0',
       grad_fn=<AddBackward0>)
tensor([-0.1459, -0.0978, -0.0176, -0.0727, -0.0733, -0.1363,  0.0458, -0.0753,
         0.1888, -0.0271,  0.1033, -0.2813,  0.1313,  0.2141], device='cuda:0',
       grad_fn=<AddBackward0>)
tensor([0.9621, 0.3114], device='cuda:0', grad_fn=<SliceBackward0>)


In [38]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

classifier = param_classifier(8, 2).to(device)
ode_dim = 2
integration_time = 1
# node = nODE(ode_dim, architecture='both', time_interval=[0, integration_time]).to(device)
# # replaced by classifier.neuralODE
loss_fn = torch.nn.MSELoss()

optimizer = torch.optim.Adam(classifier.parameters(), lr=1e-3)
"""
for name, param in classifier.named_parameters():
    print(name, param.requires_grad)

for name, param in node.named_parameters():
    print(name, param.requires_grad)
"""
for epoch in range(10):

    for batch in loader:
        optimizer.zero_grad()
        loss = 0

        x = batch[0][0].to(device)
        y = batch[1][0].to(device)

        # estimate parameters using classifier network
        p = classifier(x)

        for i in range(0,len(p)):
            # integrate ODE

            param = p[i]
            u0 = x[i][:2]

            ut = y[i][:2]
            A = y[i][2:]

            ut_hat = classifier.forward_integration(u0, param)

            # get adjacency matrix
            A_hat = classifier.get_adjacency(param)

            # add to loss: error between found solution at time t and true solution
            loss = loss + loss_fn(ut_hat.float(), ut.float())

            # add to loss: difference between estimated network and true network
            loss = loss + 0.01*loss_fn(A_hat.float(),A.float())

        #loss += loss_fn(p, 0*p)
        print('loss: ')
        print(loss)

        # backward propagation
        loss.backward(retain_graph=True)
        for name, param in classifier.named_parameters():
            if param.grad is None:
                print(f"Gradient for {name} is None")
            #else:
            #    # print(f"Gradient for {name}: {param.grad}")
        optimizer.step()
    # print(list(classifier.parameters())[0].grad)


loss: 
tensor(23.1476, device='cuda:0', grad_fn=<AddBackward0>)
loss: 
tensor(21.3256, device='cuda:0', grad_fn=<AddBackward0>)
loss: 
tensor(15.4054, device='cuda:0', grad_fn=<AddBackward0>)
loss: 
tensor(19.6824, device='cuda:0', grad_fn=<AddBackward0>)
loss: 
tensor(26.8774, device='cuda:0', grad_fn=<AddBackward0>)
loss: 
tensor(23.8654, device='cuda:0', grad_fn=<AddBackward0>)
loss: 
tensor(21.7436, device='cuda:0', grad_fn=<AddBackward0>)
loss: 
tensor(29.5502, device='cuda:0', grad_fn=<AddBackward0>)
loss: 
tensor(20.5728, device='cuda:0', grad_fn=<AddBackward0>)
loss: 
tensor(19.0474, device='cuda:0', grad_fn=<AddBackward0>)
loss: 
tensor(22.3489, device='cuda:0', grad_fn=<AddBackward0>)
loss: 
tensor(40.1671, device='cuda:0', grad_fn=<AddBackward0>)
loss: 
tensor(30.5535, device='cuda:0', grad_fn=<AddBackward0>)
loss: 
tensor(18.0123, device='cuda:0', grad_fn=<AddBackward0>)
loss: 
tensor(25.7295, device='cuda:0', grad_fn=<AddBackward0>)
loss: 
tensor(24.6098, device='cuda:0', 

KeyboardInterrupt: 

In [ ]:
correct = 0
total = 0
for i in range(0, len(x_train)):
    x = x_train[i][0].to(device)
    y = y_train[i][0].to(device)
    out = classifier(x)

    w_hat = classifier.get_adjacency(out)
    w = y[2:]

    values_log_W = torch.sort(torch.log(torch.abs(w_hat)))[0]
    threshold = values_log_W[torch.sort(values_log_W[1:] - values_log_W[:-1])[1][-1] + 1]
    W_bool_activation = (w_hat >= torch.exp(threshold))
    W_bool_repr = (w_hat <= -torch.exp(threshold))
    a_hat = W_bool_activation.float() - W_bool_repr.float()

    values_log_W = torch.sort(torch.log(torch.abs(w)))[0]
    threshold = values_log_W[torch.sort(values_log_W[1:] - values_log_W[:-1])[1][-1] + 1]
    W_bool_activation = (w >= torch.exp(threshold))
    W_bool_repr = (w <= -torch.exp(threshold))
    a = W_bool_activation.float() - W_bool_repr.float()

    print(a_hat)
    print(a)

    # guess = out.cpu() > 0.7
    # print(guess)
    # print(data_point[1])
    if sum(abs(a_hat.float()-a)) < 0.001:
        correct += 1
    total += 1

print(correct/total)

tensor([ 0., -1., -1., -1.], device='cuda:0')
tensor([1., 1., 1., 0.], device='cuda:0')
tensor([ 0., -1.,  0., -1.], device='cuda:0')
tensor([0., 0., 0., 0.], device='cuda:0')
tensor([-1.,  0.,  1., -1.], device='cuda:0')
tensor([0., 0., 0., 0.], device='cuda:0')
tensor([1., 0., 1., 1.], device='cuda:0')
tensor([0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 1., 0.], device='cuda:0')
tensor([0., 0., 1., 0.], device='cuda:0')
tensor([ 0.,  0.,  0., -1.], device='cuda:0')
tensor([0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 1., 0.], device='cuda:0')
tensor([0., 0., 0., 0.], device='cuda:0')
tensor([ 0.,  0.,  1., -1.], device='cuda:0')
tensor([ 0., -1.,  0.,  0.], device='cuda:0')
tensor([0., 0., 1., 1.], device='cuda:0')
tensor([0., 0., 0., 0.], device='cuda:0')
tensor([ 0.,  0.,  0., -1.], device='cuda:0')
tensor([0., 0., 0., 0.], device='cuda:0')
tensor([ 0.,  0.,  1., -1.], device='cuda:0')
tensor([0., 1., 0., 0.], device='cuda:0')
tensor([0., 0., 1., 0.], device='cuda:0')
te